In [ ]:
import os
import requests
from ddgs import DDGS
from time import sleep
from tqdm import tqdm
import random
import kagglehub
import shutil
dataset_folder = "ddg_custom_bmw_dataset" #output downloaded imgs

def add_non_bmw():
    #first using stanford cars dataset to add non bmw cars
    print("grabbing stanford cars dataset by jkrause / jutrera")
    path = kagglehub.dataset_download("jutrera/stanford-car-dataset-by-classes-folder")
    stanford_src = os.path.join(path, 'car_data', 'car_data', 'train')

    destination_directory = os.path.join(dataset_folder, 'non_bmw_cars')
    if not os.path.exists(destination_directory):
        os.makedirs(destination_directory)

    non_bmw_folders = [f for f in os.listdir(stanford_src) if "BMW" not in f and "bmw" not in f]
    count = 0
    while count <500: 
        folder = random.choice(non_bmw_folders)
        files = os.listdir(os.path.join(stanford_src, folder))
        if not files: continue
        image_name = random.choice(files)

        src = os.path.join(stanford_src, folder, image_name)
        dst = os.path.join(destination_directory, f"other_car_{count}.jpg")
        if not os.path.exists(dst):
            shutil.copyfile(src, dst)
            count+=1
    print("500 non bmw cars added to non_bmw_cars folder")
    

    #next using ddgs to add non cars
    non_car_queries = ["forest landscape", 
                       "city street people", 
                       "plate of food", 
                       "living room interior", 
                       "cat and dog", 
                       "mountain scenery", 
                       "beach sunset", 
                       "computer on desk", 
                       "bird in tree", 
                       "ocean waves", 
                       "snowy mountain", 
                       "desert landscape", 
                       "rainy city street", 
                       "tropical island", 
                       "night sky stars", 
                       "busy marketplace", 
                       "ancient ruins", 
                       "colorful hot air balloons", 
                       "wild animals safari",
                       "house exterior",
                       "kitchen appliances",
                       "musical instruments",
                       "sports equipment",
                       "children playing",
                       "office workspace",
                       "gardening tools",
                       "underwater coral reef",
                       "festival celebration",
                       "historical monument",
                       "fashion accessories"]

    # clean folder name
    destination_directory = os.path.join(dataset_folder, 'non_cars')
    if not os.path.exists(destination_directory):
        os.makedirs(destination_directory)
    limit_per_query = 35
    
    for query in tqdm(non_car_queries, desc="Non-Car Queries"):
        prefix = query.replace(" ", "_")
        existing_images = len([f for f in os.listdir(destination_directory) if f.startswith(prefix)])
        if existing_images >= limit_per_query:
            continue
        needed = limit_per_query - existing_images
    
    # try 3 times before giving up on a car
        for attempt in range(3):
            try:
                with DDGS() as ddgs:
                    results = ddgs.images(
                        query, 
                        region="wt-wt", 
                        safesearch="off", 
                        max_results=100 #a lot of the links must not have valid imgs because its not adding 500 imgs
                        #so looking for tons of links to solve this
                    )
                    # progress bar for individual car model
                    with tqdm(total=needed, desc=f"--> {query}", unit="img", leave=False) as pbar:
                        count = 0
                        for r in results:
                            if count >= needed:
                                break
                            
                            url = r['image']
                            try:
                                response = requests.get(url, timeout=3)
                                if response.status_code == 200:
                                    # save with unique name to prevent overwrites
                                    # existing count + new count
                                    filename = f"{prefix}_{existing_images + count:03d}.jpg"
                                    with open(os.path.join(destination_directory, filename), 'wb') as f:
                                        f.write(response.content)
                                    
                                    count += 1
                                    pbar.update(1)
                            except:
                                pass
                
                # if finish loop without crashing, stop retrying
                tqdm.write(f" Finished!! {query} (+{count} new images)")
                break 

            except Exception as e:
                tqdm.write(f" error on {query}: {e}")
                tqdm.write("   waiting 10 seconds before retrying...")
                sleep(10) # retry

if __name__ == "__main__":
    print(f" starting/resuming download")
    add_non_bmw()
    

/Users/jib/Documents/VSCODE/bmwgpt/.venv/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 starting/resuming download
grabbing stanford cars dataset by jkrause / jutrera
500 non bmw cars added to non_bmw_cars folder


Non-Car Queries:   3%|▎         | 1/30 [00:42<20:45, 42.95s/it]

 Finished!! forest landscape (+35 new images)


Non-Car Queries:   7%|▋         | 2/30 [00:56<12:05, 25.91s/it]

...

 Finished!! festival celebration (+35 new images)


Non-Car Queries:  97%|█████████▋| 29/30 [11:19<00:31, 31.38s/it]

 Finished!! historical monument (+35 new images)


Non-Car Queries: 100%|██████████| 30/30 [11:43<00:00, 23.43s/it]

 Finished!! fashion accessories (+35 new images)
